In [9]:
import numpy as np
import pandas as pd
import os
import random

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset

from transformers import (
    DataCollatorWithPadding,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    get_linear_schedule_with_warmup
)
from datasets import Dataset

from sklearn.preprocessing import MinMaxScaler

os.chdir("/g/data/jr19/rh2942/text-empathy/")
from evaluation import pearsonr
from utils.utils import plot, get_device, set_all_seeds
from utils.common import EarlyStopper

In [10]:
os.environ['TOKENIZERS_PARALLELISM'] = 'false' # due to huggingface warning
os.environ['TRANSFORMERS_NO_ADVISORY_WARNINGS'] = 'true'

In [11]:
class RobertaRegressor(nn.Module):
    def __init__(self, checkpoint):
        super(RobertaRegressor, self).__init__()
        self.transformer = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=768)
        self.fc1 = nn.Sequential(
            nn.Linear(768, 512), nn.Tanh(), nn.Dropout(0.2)
        )
        self.fc2 = nn.Sequential(
            nn.Linear(512+5, 256), nn.Tanh(), nn.Dropout(0.2),
            nn.Linear(256, 1)
        )

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        gender=None,
        education=None,
        race=None,
        age=None,
        income=None
    ):

        output = self.transformer(
            input_ids= input_ids,
            attention_mask=attention_mask,
        )

        output = self.fc1(output.logits)
        output = torch.cat([output, gender, education, race, age, income], 1)
        output = self.fc2(output)
        return output

In [12]:
class DataModule:
    def __init__(self, task, checkpoint, batch_size, feature_to_tokenise, seed):

        self.task = task
        self.checkpoint = checkpoint
        self.batch_size = batch_size
        self.tokeniser = AutoTokenizer.from_pretrained(
            self.checkpoint,
            use_fast=True
        )
        self.data_collator = DataCollatorWithPadding(tokenizer=self.tokeniser)
        self.feature_to_tokenise = feature_to_tokenise # to tokenise function
        self.seed = seed

        assert len(self.task) == 2, 'task must be a list with two elements'
    
    def _process_raw(self, path, send_label):
        data = pd.read_csv(path, sep='\t')
    
        if send_label:
            text = data[self.feature_to_tokenise + self.task]
        else:
            text = data[self.feature_to_tokenise]

        demog = ['gender', 'education', 'race', 'age', 'income']
        data_demog = data[demog]
        scaler = MinMaxScaler()
        data_demog = pd.DataFrame(
            scaler.fit_transform(data_demog),
            columns=demog
        )
        data = pd.concat([text, data_demog], axis=1) 
        return data

    def _tokeniser_fn(self, sentence):
        if len(self.feature_to_tokenise) == 1: # only one feature
            return self.tokeniser(sentence[self.feature_to_tokenise[0]], truncation=True)
        # otherwise tokenise a pair of sentence
        return self.tokeniser(sentence[self.feature_to_tokenise[0]], sentence[self.feature_to_tokenise[1]], truncation=True)

    def _process_input(self, file, send_label):
        data = self._process_raw(path=file, send_label=send_label)
        data = Dataset.from_pandas(data, preserve_index=False) # convert to huggingface dataset
        data = data.map(self._tokeniser_fn, batched=True, remove_columns=self.feature_to_tokenise) # tokenise
        data = data.with_format('torch')
        return data

    # taken from https://pytorch.org/docs/stable/notes/randomness.html
    def _seed_worker(self, worker_id):
        worker_seed = torch.initial_seed() % 2**32
        np.random.seed(worker_seed)
        random.seed(worker_seed)     

    def dataloader(self, file, send_label, shuffle):
        data = self._process_input(file=file, send_label=send_label)

        # making sure the shuffling is reproducible
        g = torch.Generator()
        g.manual_seed(self.seed)
        
        return DataLoader(
            data,
            batch_size=self.batch_size,
            shuffle=shuffle,
            collate_fn=self.data_collator,
            num_workers=12,
            worker_init_fn=self._seed_worker,
            generator=g
        )

In [13]:
class Trainer:
    def __init__(self, task, checkpoint, lr, n_epochs, train_loader,
                 dev_loader, dev_label_crowd, dev_label_gpt, device_id,
                 anno_diff, first_fine_tune):
        self.device = get_device(device_id)
        self.task = task
        self.checkpoint = checkpoint
        self.first_fine_tune = first_fine_tune
        
        self.model = RobertaRegressor(checkpoint=self.checkpoint).to(self.device)
    
        self.lr = lr
        self.n_epochs = n_epochs
        self.train_loader = train_loader
        self.dev_loader = dev_loader
        self.dev_label_crowd = dev_label_crowd
        self.dev_label_gpt = dev_label_gpt
        
        self.loss_fn = nn.MSELoss()
        self.optimiser = torch.optim.AdamW(
            params=self.model.parameters(),
            lr=self.lr,
            betas=(0.9, 0.98),
            eps=1e-06,
            weight_decay=0.1
        )

        n_training_step = self.n_epochs * len(self.train_loader)
        self.lr_scheduler = get_linear_schedule_with_warmup(
            optimizer=self.optimiser,
            num_warmup_steps=0.06*n_training_step,
            num_training_steps=n_training_step
        )
        
        self.best_pearson_r = -1.0 # initiliasation
        self.early_stopper = EarlyStopper(patience=3, min_delta=0.01)
        
        self.anno_diff = anno_diff
        
        assert len(self.task) == 2, 'task must be a list with two elements'

    def _label_fix(self, crowd, gpt):
        condition = torch.abs(crowd.detach() - gpt.detach()) > self.anno_diff
        crowd[condition] = gpt[condition]
        return crowd

    def _training_step(self):
        tr_loss = 0.0
        self.model.train()
    
        for data in self.train_loader:
            input_ids = data['input_ids'].to(self.device, dtype=torch.long)
            attention_mask = data['attention_mask'].to(self.device, dtype=torch.long)
            
            gpt = data[self.task[0]].to(self.device, dtype=torch.float).view(-1, 1)
            targets = data[self.task[1]].to(self.device, dtype=torch.float).view(-1, 1)
            
            gender = data['gender'].to(self.device, dtype=torch.float).view(-1, 1)
            education = data['education'].to(self.device, dtype=torch.float).view(-1, 1)
            race = data['race'].to(self.device, dtype=torch.float).view(-1, 1)
            age = data['age'].to(self.device, dtype=torch.float).view(-1, 1)
            income = data['income'].to(self.device, dtype=torch.float).view(-1, 1)
    
            outputs = self.model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                gender=gender,
                education=education,
                race=race,
                age=age,
                income=income
            )

            targets = self._label_fix(crowd=targets, gpt=gpt)
            
            loss = self.loss_fn(outputs, targets)
            tr_loss += loss.item()
    
            self.optimiser.zero_grad()
            loss.backward()
            self.optimiser.step()
            self.lr_scheduler.step()

        epoch_loss = tr_loss / len(train_loader)
        print(f'Train loss: {epoch_loss}')

    def fit(self, save_model=False):
        dev_label_anno = pd.read_csv(self.dev_label_gpt, sep='\t')
        true_gpt = dev_label_anno.loc[:, 'empathy'].tolist()
        dev_label_crowd = pd.read_csv(self.dev_label_crowd, sep='\t', header=None)
        true = dev_label_crowd.iloc[:, 0].tolist()

        # gpt-label on the second fine tune 
        true = self._label_fix(crowd=torch.tensor(true), gpt=torch.tensor(true_gpt))
        true = true.tolist()

        for epoch in range(self.n_epochs):
            print(f'Epoch: {epoch+1}')
            self._training_step()

            preds = self.evaluate(dataloader=self.dev_loader, load_model=False)
            
            pearson_r = pearsonr(true, preds)
            print(f'Pearson r: {pearson_r}')
            
            val_loss = self.loss_fn(torch.tensor(preds), torch.tensor(true))
            print('Validation loss:', val_loss.item())
            
            if self.early_stopper.early_stop(val_loss):
                break

            if (pearson_r > self.best_pearson_r):
                self.best_pearson_r = pearson_r

                # save
                if save_model:
                    torch.save(self.model.state_dict(), self.checkpoint+'-gpt-finetuned.pth')
                    print("Saved the second finetuned model in epoch " + str(epoch+1))
            
            print(f'Best dev set Pearson r: {self.best_pearson_r}\n')

    def evaluate(self, dataloader, load_model=False):
        if load_model:
            self.model.load_state_dict(torch.load(self.checkpoint+'-gpt-finetuned.pth'))
    
        pred = torch.empty((len(dataloader.dataset), 1), device=self.device) # len(self.dev_loader.dataset) --> # of samples
        self.model.eval()
    
        with torch.no_grad():
            idx = 0
            for data in dataloader:
                input_ids = data['input_ids'].to(self.device, dtype=torch.long)
                attention_mask = data['attention_mask'].to(self.device, dtype=torch.long)
                gender = data['gender'].to(self.device, dtype=torch.float).view(-1, 1)
                education = data['education'].to(self.device, dtype=torch.float).view(-1, 1)
                race = data['race'].to(self.device, dtype=torch.float).view(-1, 1)
                age = data['age'].to(self.device, dtype=torch.float).view(-1, 1)
                income = data['income'].to(self.device, dtype=torch.float).view(-1, 1)
        
                outputs = self.model(input_ids, attention_mask, gender, education, race, age, income)

                batch_size = outputs.shape[0]
                pred[idx:idx+batch_size, :] = outputs
                idx += batch_size
            
        return [float(k) for k in pred]   

In [14]:
checkpoint = 'roberta-base'
task = ['empathy', 'wrong_empathy']
# feature_to_tokenise=['demographic_essay', 'article']
# feature_to_tokenise=['demographic', 'essay']
feature_to_tokenise=['demographic_essay']
seed = 0

################# COMBINED TRAIN FILE ##############
# train_file = './data/WS22-WS23-train-gpt.tsv'
train_file = './data/WS22-WS23-sep-from-aug-train-gpt.tsv'
# train_file = './data/essay-train-ws22-ws23.tsv'
# train_file = './data/PREPROCESSED-WS22-WS23-train.tsv'
# train_file = './data/COMBINED-PREPROCESSED-PARAPHRASED-WS22-WS23-train.tsv'

################# WASSA 2022 ####################
# dev_file = './data/PREPROCESSED-WS22-dev.tsv'
# dev_label_file = './data/WASSA22/goldstandard_dev_2022.tsv'
# test_file = './data/PREPROCESSED-WS22-test.tsv'

##### GPT annotation
# train_file = './data/WS22-train-gpt.tsv'
# dev_file = './data/WS22-dev-gpt.tsv'
# dev_label_file = './data/WS22-dev-gpt.tsv'


################# WASSA 2023 ####################
# dev_file = './data/PREPROCESSED-WS23-dev.tsv'
dev_label_crowd = './data/WASSA23/goldstandard_dev.tsv'
test_file = './data/PREPROCESSED-WS23-test.tsv'

##### GPT annotation
# train_file = './data/WS23-train-gpt.tsv'
dev_file = './data/WS23-dev-gpt.tsv'
dev_label_gpt = './data/WS23-dev-gpt.tsv'

In [15]:
set_all_seeds(seed)

data_module = DataModule(
    task=task,
    checkpoint=checkpoint,
    batch_size=16,
    feature_to_tokenise=feature_to_tokenise,
    seed=0
)

train_loader = data_module.dataloader(file=train_file, send_label=True, shuffle=True)
dev_loader = data_module.dataloader(file=dev_file, send_label=False, shuffle=False)
test_loader = data_module.dataloader(file=test_file, send_label=False, shuffle=False)

trainer = Trainer(
    task=task,
    checkpoint=checkpoint,
    lr=1e-5,
    n_epochs=10,
    train_loader=train_loader,
    dev_loader=dev_loader,
    dev_label_gpt=dev_label_gpt,
    dev_label_crowd=dev_label_crowd,
    device_id=0,
    anno_diff=1.5,
    first_fine_tune=False
)

trainer.fit(save_model=False)

Map:   0%|          | 0/2634 [00:00<?, ? examples/s]

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 1
Train loss: 13.341799072063331
Pearson r: 0.209
Validation loss: 2.5673940181732178
Best dev set Pearson r: 0.209

Epoch: 2
Train loss: 2.7105073563980335
Pearson r: 0.617
Validation loss: 1.3849575519561768
Best dev set Pearson r: 0.617

Epoch: 3
Train loss: 1.7792145237778172
Pearson r: 0.658
Validation loss: 1.192095398902893
Best dev set Pearson r: 0.658

Epoch: 4
Train loss: 1.3710925178094344
Pearson r: 0.709
Validation loss: 1.099982738494873
Best dev set Pearson r: 0.709

Epoch: 5
Train loss: 1.1807976664918842
Pearson r: 0.699
Validation loss: 1.0979441404342651
Best dev set Pearson r: 0.709

Epoch: 6
Train loss: 0.9269254198580077
Pearson r: 0.71
Validation loss: 1.0793267488479614
Best dev set Pearson r: 0.71

Epoch: 7
Train loss: 0.7482946957602645
Pearson r: 0.713
Validation loss: 1.0665911436080933
Best dev set Pearson r: 0.713

Epoch: 8
Train loss: 0.6437775433966608
Pearson r: 0.713
Validation loss: 1.090736985206604
Best dev set Pearson r: 0.713

Epoch: 9
Trai

# Test

In [15]:
pred = trainer.evaluate(dataloader=test_loader, load_model=True)
pred_df = pd.DataFrame({'emp': pred, 'dis': pred}) # we're not predicting distress, just aligning with submission system
pred_df.to_csv('./tmp/predictions_EMP.tsv', sep='\t', index=None, header=None)
pred_df

,emp,dis
0,3.738305,3.738305
1,6.001295,6.001295
2,5.443245,5.443245
3,2.081530,2.081530
4,5.864893,5.864893
...,...,...
95,5.833966,5.833966
96,4.599444,4.599444
97,5.307490,5.307490
98,6.035558,6.035558
